# Write a program to remove left factoring from a given grammar.

## Left Factoring

In [1]:
from itertools import takewhile 
epsilon = '\u03B5'

In [2]:
def findPrefix(prod):
    dic = { a[0] : [] for a in prod}
    for a in prod:
        dic[a[0]].append(a)
    prefix = dict()
    for (a,b) in dic.items():
        pre = ''.join(c[0] for c in takewhile(lambda x: all(x[0] == y for y in x), zip(*b)))
        prefix[pre] = b
    return prefix

In [3]:
def LeftFactor(Grammar):
    nt = list(set([t[0] for t in Grammar]))
    newGrammar = Grammar.copy()
    for i in nt:
        prod = [t[1] for t in Grammar if t[0] == i and t[1] != '']
        prefix = findPrefix(prod)
        for (a,b) in prefix.items():
            if len(b) > 1:  
                new = i + '\''
                new_prod = a+new
                newGrammar.append((i,new_prod))
                for p in b:
                    newGrammar.remove((i,p))
                    newGrammar.append((new,p[len(a):]))
    return newGrammar

In [4]:
def printGrammar(Grammar):
    lhs = list(set([t[0] for t in Grammar]))
    for l in lhs:
        rhs = [t[1] for t in Grammar if t[0] == l]
        if '' in rhs :
            rhs.remove('')
            rhs.append(epsilon)
        b = ' | '.join(rhs)
        print('{0:<3} -> {1:<25}'.format(l,b))

In [5]:
def RemoveLeftFactoring(Grammar):
    change = True
    print("Original Grammar : \n")
    printGrammar(Grammar)
    old = Grammar.copy()
    while change == True:
        new = LeftFactor(old)
        change = not(old == new)
        old = new.copy()
    print("\nLeft Factored Grammar : \n")
    printGrammar(new)  

In [6]:
Grammar = [('A','aAB'),('A','aBc'),('A','aAc')]
RemoveLeftFactoring(Grammar)

Original Grammar : 

A   -> aAB | aBc | aAc          

Left Factored Grammar : 

A'  -> Bc | AA''                
A   -> aA'                      
A'' -> B | c                    


In [7]:
Grammar = [('S','iEtS'),('S','iEtSeS'),('S','a'),('E','b')]
RemoveLeftFactoring(Grammar)

Original Grammar : 

E   -> b                        
S   -> iEtS | iEtSeS | a        

Left Factored Grammar : 

S'  -> eS | ε                   
E   -> b                        
S   -> a | iEtSS'               


# Removing Left Recursion

In [8]:
def DirectLeftRecursion(Grammar):
    nt = list(set([t[0] for t in Grammar]))
    newGrammar = Grammar.copy()
    for i in nt:
        lr = [b for a,b in Grammar if a == i and b.startswith(a)]
        nlr = [b for a,b in Grammar if a == i and b not in lr]
        if len(lr) != 0:
            new = i + '\''
            newGrammar.append((new,''))
            for j in lr:
                newGrammar.remove((i,j))
                newGrammar.append((new,j.lstrip(i) + new))
            for j in nlr:
                newGrammar.remove((i,j))
                newGrammar.append((i,j + new))
    return newGrammar

In [9]:
def IndirectLeftRecursion(Grammar):
    nt = list(set([t[0] for t in Grammar]))
    n = len(nt)
    for i in range(n):
        prod = [t[1] for t in Grammar if t[0] == nt[i]]
        for j in range(i):
            for p in prod:
                if p.startswith(nt[j]):
                    x = p.lstrip(nt[j])
                    Grammar.remove((nt[i],p))
                    pr = [t[1] for t in Grammar if t[0] == nt[j]]
                    for q in pr:
                        Grammar.append((nt[i],q+x))
        Grammar = DirectLeftRecursion(Grammar)
    return Grammar

In [10]:
def RemoveLeftRecursion(Grammar):
    change = True
    print("Original Grammar : \n")
    printGrammar(Grammar)
    old = Grammar.copy()
    while change == True:
        new = IndirectLeftRecursion(old)
        change = not(old == new)
        old = new.copy()
    print("\nAfter removing Left Recursion : \n")
    printGrammar(new)  

In [11]:
Grammar = [('A','Br'),('B','Cd'),('C','At')]
#Grammar = [('S','Sa'),('S','Sb'),('S','c'),('S','d')]
RemoveLeftRecursion(Grammar)

Original Grammar : 

C   -> At                       
B   -> Cd                       
A   -> Br                       

After removing Left Recursion : 

C   -> At                       
A'  -> tdrA' | ε                
B   -> Atd                      


In [12]:
G = [('S','ACB'),('S','Cbb'),('S','Ba'),('A','da'),('A','BC'),('B',''),('B','g'),('C',''),('C','h')]


In [13]:
RemoveLeftFactoring(G)

Original Grammar : 

C   -> h | ε                    
S   -> ACB | Cbb | Ba           
A   -> da | BC                  
B   -> g | ε                    

Left Factored Grammar : 

C   -> h | ε                    
S   -> ACB | Cbb | Ba           
A   -> da | BC                  
B   -> g | ε                    


In [14]:
RemoveLeftRecursion(G)

Original Grammar : 

C   -> h | ε                    
S   -> ACB | Cbb | Ba           
A   -> da | BC                  
B   -> g | ε                    

After removing Left Recursion : 

C   -> h | ε                    
S   -> ACB | Ba | bb | hbb      
A   -> da | BC                  
B   -> g | ε                    
